Plot the SST in the Canary Current Upwelling System.

In [85]:
import os
import glob
import filament
import datetime
import netCDF4
import cmocean
import logging
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from importlib import reload
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.mpl.ticker as cartopyticker
myproj = ccrs.PlateCarree()
coast = cfeature.GSHHSFeature(scale="f")
reload(filament)

<module 'filament' from '/home/ctroupin/Projects/FNRS-UpwellingFilaments/Filament-Submesoscale/python/filament.py'>

In [86]:
logger = logging.getLogger("filamentPlot")
logger.setLevel(logging.INFO)
logging.info("Starting")
plotvis = 0

In [87]:
datadir = "/data/SST/CapeGhir/2plot/"
datadir = "/data/OceanColor/"
figdir = "../figures/SST/"
figdir2 = "/data/SST/CapeGhir/figures/medium"
swotfile = "/data/SWOT/swot292_science.txt"
visibledir = "/data/Visible/"
datafilelist = sorted(glob.glob(os.path.join(datadir, "SNPP*.nc")))
nfiles = len(datafilelist)
logger.info("Processing {} files".format(nfiles))
mediumdomain = (-19., -6., 25., 35.)
smalldomain = (-15., -9., 27., 33.)
tinydomain = (-12., -9., 29.5, 32.)
canarydomain = (-20, -10, 25., 32)

INFO:filamentPlot:Processing 12 files


In [88]:
if not(os.path.exists(figdir)):
    os.makedirs(figdir)
if not(os.path.exists(figdir2)):
    os.makedirs(figdir2)

## Using Cartopy

In [89]:
for datafile in datafilelist:
    print(datafile)
    with netCDF4.Dataset(datafile) as nc:
        proclevel = nc.processing_level
    sst = filament.SST()
    
    # Select the reader according to processing level
    if proclevel == "L3 Mapped":
        logger.info("Level-3 data")
        sst.read_from_oceancolorL3(datafile, coordinates)
    elif proclevel == "L2":
        logger.info("Level-2 data")
        sst.read_from_oceancolorL2(datafile)
        sst.field = np.ma.masked_where(sst.qflag > 2, sst.field)
        
    figname = os.path.join(figdir, sst.get_figname())

    fig = plt.figure(figsize=(8, 6))
    ax = plt.subplot(111, projection=myproj)
    sst.add_to_plot(fig, ax, domain=tinydomain, cmap=plt.cm.RdYlBu_r,
                    clim=[21., 24.], vis=False,
                    cbarloc=[0.57, 0.23, 0.3, 0.015])
    ax.add_feature(coast, linewidth=.2, color=".5")
    filament.decorate_map(ax, canarydomain, np.arange(-20., -10., 2.), 
                              np.arange(25., 32., 1.))
    plt.savefig(figname, dpi=300, bbox_inches="tight")
    # plt.show()
    plt.close()

INFO:filamentPlot:Level-2 data


/data/OceanColor/SNPP_VIIRS.20200501T025400.L2.SST.NRT.nc


INFO:filamentPlot:Level-2 data


/data/OceanColor/SNPP_VIIRS.20200501T141201.L2.SST.NRT.nc


INFO:filamentPlot:Level-2 data


...

INFO:filamentPlot:Level-2 data


/data/OceanColor/SNPP_VIIRS.20200603T135400.L2.SST.NRT.nc


INFO:filamentPlot:Level-2 data


/data/OceanColor/SNPP_VIIRS.20200604T021800.L2.SST3.NRT.nc


TypeError: '>' not supported between instances of 'netCDF4._netCDF4.Variable' and 'int'

In [75]:
datafile = "/data/OceanColor/SNPP_VIIRS.20200604T021800.L2.SST3.NRT.nc"
#sst0 = filament.SST()
#sst0.read_from_oceancolorL2(datafile)
with netCDF4.Dataset(datafile) as nc:
    print(nc.groups['geophysical_data'].variables['sst_triple'])
    sstvar = nc.get_variables_by_attributes(standard_name="sea_surface_temperature")
    print(sstvar[:])

[]


## Using Basemap

In [5]:
coordinates = (-19., -6., 25., 35.)
m = Basemap(projection='merc', llcrnrlon=coordinates[0], llcrnrlat=coordinates[2],
            urcrnrlon=coordinates[1], urcrnrlat=coordinates[3],
            epsg=4326, resolution='h')

In [10]:
for datafile in datafilelist:
    
    sst = filament.SST()
    
    # Select the reader according to processing level
    if proclevel == "L3 Mapped":
        logger.info("Level-3 data")
        sst.read_from_oceancolorL3(datafile, coordinates)
    elif proclevel == "L2":
        logger.info("Level-2 data")
        sst.read_from_oceancolorL2(datafile)
        sst.field = np.ma.masked_where(sst.qflag > 2, sst.field)
    
    # Plot without visible
    figname = os.path.join(figdir, sst.get_figname())
    sst.make_plot(m, figname=figname, vmin=16., vmax=21, shrink=0.7)
    
    # Plot quality flag 
    figname = os.path.join(figdir, sst.get_figname() + "_qf")
    sst.make_plot_qf(m, figname=figname, shrink=0.7)
    
    
    # With visible image background
    visiblefile = os.path.join(visibledir, "VIIRS{}.tiff".format(sst.date.strftime("-%Y-%m-%dT00_00_00Z")))
    if os.path.exists(visiblefile):
        visibleim = filament.Visible()
        visibleim.read_from(visiblefile)
        visibleim.extract_area(coordinates)
    figname = os.path.join(figdir, "_".join((sst.get_figname(), "visible")))
    #sst.make_plot2(m, figname=figname, visibleim=visibleim, vmin=16., vmax=21.)

INFO:filamentPlot:Level-2 data


NameError: name 'm' is not defined

## GHRSST

In [7]:
datadir2 = "/data/SST/GHRSST/"
datafilelist2 = sorted(glob.glob(os.path.join(datadir2, "20170904*.nc")))
nfiles = len(datafilelist2)
logger.info("Processing {} files".format(nfiles))

INFO:filamentPlot:Processing 1 files


In [8]:
for datafile in datafilelist2:
    logger.info(datafile)
    sst = filament.SST()
    sst.read_from_ghrsst(datafile)
    sst.field = np.ma.masked_where(sst.qflag != 5, sst.field)
    
    # Plot without visible
    figname = os.path.join(figdir, sst.get_figname())
    sst.make_plot2(m, figname=figname)
    
    # With visible image background
    visiblefile = os.path.join(visibledir, "VIIRS_{}.tiff".format(sst.date.strftime("%Y%m%d")))
    if os.path.exists(visiblefile):
        visibleim = filament.Visible()
        visibleim.read_from(visiblefile)
        visibleim.extract_area(coordinates)
    figname = os.path.join(figdir, "_".join((sst.get_figname(), "visible")))
    sst.make_plot2(m, figname=figname, visibleim=visibleim)
    
    # With visible image background and SWOT
    swot = filament.Swot()
    swot.read_from(swotfile)
    swot.select_domain(coordinates)
    visiblefile = os.path.join(visibledir, "VIIRS_{}.tiff".format(sst.date.strftime("%Y%m%d")))
    if os.path.exists(visiblefile):
        visibleim = filament.Visible()
        visibleim.read_from(visiblefile)
        visibleim.extract_area(coordinates)
    figname = os.path.join(figdir, "_".join((sst.get_figname(), "visible", "swot")))
    sst.make_plot2(m, figname=figname, visibleim=visibleim, swot=swot)

INFO:filamentPlot:/data/SST/GHRSST/20170904140000-OSPO-L2P_GHRSST-SSTskin-VIIRS_NPP-ACSPO_V2.40-v02.0-fv01.0.nc


TypeError: '>=' not supported between instances of 'NoneType' and 'float'